In [1]:
#!sudo apt-get install -y xvfb ffmpeg
!pip install atari_py
!pip install gym[atari]
!pip install imageio
!pip install PILLOW
!pip install pyglet
!pip install typing-extensions==3.7.4.3
!pip install pyvirtualdisplay
!pip install tf-agents
!pip install ../../OSAR-keras/.
# !pip install git+https://github.com/ustyuzhaninky/OSAR-keras

Processing d:\jorgen\projects\python\phdsub\towerproject\osar-keras
  Created wheel for OSAR: filename=OSAR-0.1.7-py3-none-any.whl size=67064 sha256=1bd70c5989704d073b70887b0cef5b55c0d1ee70a24514f8a246d8175672eaee
  Stored in directory: c:\users\jorgen\appdata\local\pip\cache\wheels\df\60\40\81c908253910a541e172b609cc54a7a5400e85a6fba88b292e
Successfully built OSAR
  Attempting uninstall: OSAR
    Found existing installation: OSAR 0.1.7
    Uninstalling OSAR-0.1.7:
      Successfully uninstalled OSAR-0.1.7


In [2]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents import agents
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

from OSAR import OSARNetwork, Runner, TrialAgent

In [3]:
env_names = [
    'AirRaid-ram-v0',
    'Solaris-ram-v0',
    'Alien-ram-v0',
    'Berzerk-ram-v0',
    'Asteroids-ram-v0',
    'Venture-ram-v0',
    'MontezumaRevenge-ram-v0',
    'Asteroids-ram-v0',
    'BattleZone-ram-v0',
    'Enduro-ram-v0',
    'Tutankham-ram-v0',
    'WizardOfWor-ram-v0'
]

In [4]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 1000  # @param {type:"integer"}

batch_size = 1  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 1  # @param {type:"integer"}
memory_len = 10 # @param {type: "integer"}
n_turns = 5 # @param {type: "integer"}
num_atoms = 21  # @param {type:"integer"}
q_value = 10  # @param {type:"integer"}
n_step_update = 2  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}


In [5]:
fc_layer_params = (10,)
from tf_agents.networks import categorical_q_network, q_network

agent_specs = {
    'batch_size': 1,
    'memory_len': 2,
    'n_turns': 3,
    'fc_layer_params': fc_layer_params,
    'num_atoms': 21,
    'conv_type': '1d',
    'learning_rate': 1e-3,
    'q_value': 10,
    'n_step_update': 2,
}

def osar_generator(
    observation_spec,
    action_spec,
    batch_size,
    memory_len,
    n_turns,
    fc_layer_params,
    num_atoms,
    conv_type,
    learning_rate,
    time_step_spec,
    n_step_update,
    q_value=10,
    ):
    q_net = OSARNetwork(
        observation_spec,
        action_spec,
        batch_size,
        memory_len,
        n_turns,
        fc_layer_params=fc_layer_params,
        num_atoms=num_atoms,
        conv_type='1d',
        )
#     q_net = categorical_q_network.CategoricalQNetwork(
#     observation_spec,
#     action_spec,
#     num_atoms=num_atoms,
#     fc_layer_params=fc_layer_params)


#     class DummyNetwork(q_network.QNetwork):
#         def call(self,
#              observation,
#              reward=tf.constant([0.0], dtype=tf.float32),
#              step_type=None,
#              network_state=(),
#              training=False):
#             return super(DummyNetwork, self).call(
#                         observation,
#                         step_type=step_type,
#                         network_state=network_state,
#                         training=training
#                     )
#     q_net = DummyNetwork(
#         observation_spec,
#         action_spec,
#         fc_layer_params=fc_layer_params,
#     )
    
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

    train_step_counter = tf.Variable(0)
    
    agent = TrialAgent(
        time_step_spec,
        action_spec,
        network=q_net,
        optimizer=optimizer,
        td_errors_loss_fn=common.element_wise_squared_loss,
        train_step_counter=train_step_counter,
    )
    
#     agent = agents.categorical_dqn.categorical_dqn_agent.CategoricalDqnAgent(
#         time_step_spec,
#         action_spec,
#         categorical_q_network=q_net,
#         optimizer=optimizer,
#         td_errors_loss_fn=common.element_wise_squared_loss,
#         train_step_counter=train_step_counter,
#         min_q_value = -q_value,
#         max_q_value = q_value,
#         n_step_update = n_step_update,
#     )
    agent.initialize()
    
    return agent

    

In [6]:
experiment_common_specs = {
    'agent_specs': agent_specs,
    'agent_generator': osar_generator,
    'num_iterations': num_iterations,
    'initial_collect_steps': initial_collect_steps,
    'collect_steps_per_iteration': collect_steps_per_iteration,
    'replay_buffer_max_length': replay_buffer_max_length,
    'num_eval_episodes': num_eval_episodes,
    'eval_interval': eval_interval,
    'n_step_update': n_step_update,
}

In [7]:
configs = []
for name in env_names:
    config = experiment_common_specs.copy()
    config['env_name'] = name
    configs.append(config)

In [ ]:
%%time
runner = Runner(model_name='test_one', logpath='', list_configs=configs)
runner.run(progress=False, experiment_progress=True)

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.


C:\Users\Jorgen\AppData\Local\Temp\tmp0t6vm6gb.py:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
  ag__.if_stmt(ag__.not_(ag__.or_((lambda : (ag__.ld(policy_state) is None)), (lambda : ag__.or_((lambda : (ag__.ld(policy_state) is ())), (lambda : (ag__.ld(policy_state) is [])))))), if_body_2, else_body_2, get_state_2, set_state_2, (), 0)
C:\Users\Jorgen\AppData\Local\Temp\tmpr6srp0dj.py:74: SyntaxWarning: "is" with a literal. Did you mean "=="?
  ag__.if_stmt((ag__.ld(policy_state) is ()), if_body_2, else_body_2, get_state_2, set_state_2, ('retval_', 'do_return'), 2)
C:\Users\Jorgen\AppData\Local\Temp\tmphuxa6icw.py:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  retval_ = ag__.and_((lambda : (ag__.ld(state) is not None)), (lambda : ag__.and_((lambda : (ag__.ld(state) is not ())), (lambda : (ag__.ld(state) is not [])))))
  0%|                                                                                        | 0/20000 [00:00<?, ?it/s]C:\Users\Jorgen\AppD

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Episode 11:   0%|                              | 11/20000 [02:05<59:10:34, 10.66s/it, avg_return=245, train_loss=0.859]